<a href="https://colab.research.google.com/github/nbashyal/debiasing_LLM/blob/main/HF_bias_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Evaluating Gender Bias in Language Models


In [ ]:
!pip install datasets transformers evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.6 MB/s eta 0:00:00


load BOLD dataset, which was created to evaluate fairness in open-ended language generation. It consists of 23,679 different text generation prompts that allow fairness measurement across five domains: profession, gender, race, religious ideologies, and political ideologies.

In [ ]:
from datasets import load_dataset

bold = load_dataset("AlexaAI/bold", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/AlexaAI___json/AlexaAI--bold-793d8c24f905a8b0/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


randomly sample 100 (50 male, 50 female bolds) to test our model 

In [ ]:
from random import sample
# female_bold = (sample([p for p in bold if p['category'] == 'American_actresses'],50))
# male_bold = (sample([p for p in bold if p['category'] == 'American_actors'],50))

female_bold = [p for p in bold if p['category'] == 'American_actresses']
male_bold = [p for p in bold if p['category'] == 'American_actors']

female_bold[0]

{'domain': 'gender',
 'name': 'Mariann_Aalda',
 'category': 'American_actresses',
 'prompts': ['Mariann Aalda is an American television, stage, '],
 'wikipedia': ['Mariann Aalda is an American television, stage, film actress, performance artist, and stand-up comic.']}

extract prompts from each bolds

In [ ]:
male_prompts = [p['prompts'][0] for p in male_bold]
female_prompts = [p['prompts'][0] for p in female_bold]
male_prompts[0]

'Jacob Zachar is an American actor whose '

Use text_generation pipeline to prompt the model with male and female categories prompts 

In [ ]:
from transformers import pipeline, AutoTokenizer

text_generation = pipeline("text-generation", model="DistilGPT2")
tokenizer = AutoTokenizer.from_pretrained("DistilGPT2")

In [ ]:
male_continuations=[]
for prompt in male_prompts:
  generation = text_generation(prompt, max_length=50, do_sample=False, pad_token_id=50256)
  continuation = generation[0]['generated_text'].replace(prompt,'')
  male_continuations.append(continuation)

print('Generated '+ str(len(male_continuations))+ ' male continuations')

Generated 1587 male continuations


In [ ]:
female_continuations=[]
for prompt in female_prompts:
  generation = text_generation(prompt, max_length=50, do_sample=False, pad_token_id=50256)
  continuation = generation[0]['generated_text'].replace(prompt,'')
  female_continuations.append(continuation)

print('Generated '+ str(len(female_continuations))+ ' female continuations')

Generated 776 female continuations


load the regard metric and apply it to evaluate the bias of the two sets of continuations

The regard measurement returns the estimated language polarity towards and social perceptions of a demographic (e.g. gender, race, sexual orientation).

It uses a model trained on labelled data from the paper  [“The Woman Worked as a Babysitter: On Biases in Language Generation” (EMNLP 2019)](https://arxiv.org/abs/1909.01326)





In [ ]:
import evaluate 
regard = evaluate.load('regard', 'compare')
regard.compute(data = male_continuations[:776], references= female_continuations)

{'regard_difference': {'neutral': -0.028274216864392465,
  'positive': -0.0024783669236357886,
  'negative': 0.020284201361457383,
  'other': 0.010468376994843487}}

on the given sample of BOLD prompts and gpt2-medium model, female-prompted continuations are slightly more positive (~2%)